<a href="https://colab.research.google.com/github/16bitOni/Juris/blob/main/Juri_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [5]:
# prompt: write code to unzip the archive.zip and store it in data folder

import zipfile
import os

# Create the 'data' directory if it doesn't exist
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Specify the path to your zip file
zip_file_path = "archive.zip"  # Replace with the actual path to your zip file


try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)  # Extract to the 'data' directory
    print(f"Successfully extracted '{zip_file_path}' to '{data_dir}'")
except FileNotFoundError:
    print(f"Error: '{zip_file_path}' not found.")
except zipfile.BadZipFile:
    print(f"Error: Invalid zip file '{zip_file_path}'.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Successfully extracted 'archive.zip' to 'data'


In [23]:
import json
from datasets import Dataset

def load_json_data(file_paths):
    texts = []
    for file in file_paths:
        with open(file, "r") as f:
            data = json.load(f)
            for item in data:
                # Format: "Question: X Answer: Y"
                text = f"Question: {item['question']} Answer: {item['answer']}"
                texts.append(text)
    return Dataset.from_dict({"text": texts})

dataset = load_json_data([
    "data/constitution_qa.json",
    "data/crpc_qa_normalized.json",
    "data/ipc_qa.json"
])
dataset = dataset.train_test_split(test_size=0.1)

In [24]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",  # Fixed-length padding
        max_length=256,        # Adjust based on your data
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/13088 [00:00<?, ? examples/s]

Map:   0%|          | 0/1455 [00:00<?, ? examples/s]

In [26]:
def add_labels(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)

Map:   0%|          | 0/13088 [00:00<?, ? examples/s]

Map:   0%|          | 0/1455 [00:00<?, ? examples/s]

In [27]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

model = GPT2LMHeadModel.from_pretrained("gpt2")
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-legal-india",
    per_device_train_batch_size=4,  # Reduce if OOM errors occur
    num_train_epochs=3,
    logging_steps=100,
    save_steps=10_000,
    learning_rate=5e-5,
    fp16=True,
    evaluation_strategy="epoch",
    remove_unused_columns=True,  # Automatically ignores unused columns
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=collator,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.854600,1.793963
2,1.585100,1.661691
3,1.452500,1.624334


TrainOutput(global_step=9816, training_loss=1.7173132123068862, metrics={'train_runtime': 1610.009, 'train_samples_per_second': 24.387, 'train_steps_per_second': 6.097, 'total_flos': 5129685172224000.0, 'train_loss': 1.7173132123068862, 'epoch': 3.0})

In [35]:
def ask_question(question):
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,  # Limit answer length
        num_beams=5,
        early_stopping=True,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(ask_question("What is Article 21 of the Indian Constitution?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What is Article 21 of the Indian Constitution? Answer: Article 21 of the Indian Constitution states that nothing in this Constitution shall be deemed to affect the power of Parliament to make laws with respect to any of the matters enumerated in List I of the Seventh Schedule. Additionally, nothing in this Constitution shall be deemed to affect the power of Parliament to make laws with respect to any of the matters enumerated in List I of the Seventh Schedule. Additionally, nothing in this Constitution shall be deemed to affect the power of Parliament to make laws with respect to any of the matters enumerated in List I of the Seventh Schedule. Additionally, nothing in this Constitution shall be deemed to affect the power of Parliament to make laws with respect to any of the matters enumerated in List I of the Seventh Schedule. Additionally, nothing in this Constitution shall be deemed to affect the power of Parliament to make laws with respect to any of the matters enumerated